In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.optim as opt
import torch.nn as nn
import torch
from torchvision.models import vgg16

import numpy as np
from src.voc_dataset import *
from src.region_proposal import RegionProposalNetwork, get_target_weights
from src.generate_anchor_data import AnchorDataGenerator
from src.generate_anchors import generate_anchors
from src.anchor_util import generate_all_anchors
from src.proposal import ProposalGenerator

import pickle
from os import listdir
from os.path import join